# 6.3.1读取数据集

In [21]:
import torch
import random
import zipfile

f = open("../data/house_prediction/jaychou_lyrics.txt")
corpus_chars = f.read()
corpus_chars[:40]
f.close()

In [22]:
# 换行符替换成空格
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
corpus_chars = corpus_chars[0:10000]

# 6.3.2 建立字符索引

In [23]:
idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)] )
vocab_size = len(char_to_idx)
vocab_size

1027

In [25]:
# 之后，将训练数据集中每个字符转化为索引，并打印前20个字符及其对应的索引
corpus_indices = [ char_to_idx[char] for char in corpus_chars ]
sample = corpus_indices[:20]
print('chars', ''.join( [ idx_to_char[idx] for idx in sample ] ) )
print('indices:', sample)

chars 想要有直升机 想要和你飞到宇宙去 想要和
indices: [234, 585, 117, 220, 613, 261, 622, 234, 585, 233, 150, 980, 173, 115, 165, 183, 622, 234, 585, 233]


# 6.3.3 时序数据的采样
采样方式：随机采样和相邻采样

## 随机采样 

In [27]:
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    # 减一是因为输出的索引x是相应输入的索引y加1
    num_examples = (len(corpus_indices) - 1 ) // num_steps
    epoch_size = num_examples // batch_size
    exapmple_indices = list(range(num_examples))
    random.shuffle(exapmple_indices)

    #返回从pos开始的长为num_steps的序列
    def _data(pos):
        return corpus_indices[pos: pos + num_steps]
    
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    for i in range(epoch_size):
        i = i * batch_size
        batch_indices = exapmple_indices[i: i + batch_size]
        X = [ _data(j * num_steps) for j in batch_indices ]
        Y = [ _data(j * num_steps + 1)  for j in batch_indices]
        yield torch.tensor(X, dtype=torch.float32, device=device), torch.tensor(Y, dtype=torch.float32, device=device)

In [29]:
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=3):
    print('X:', X, '\nY:', Y, '\n')

X: tensor([[18., 19., 20.],
        [12., 13., 14.]]) 
Y: tensor([[19., 20., 21.],
        [13., 14., 15.]]) 

X: tensor([[ 6.,  7.,  8.],
        [21., 22., 23.]]) 
Y: tensor([[ 7.,  8.,  9.],
        [22., 23., 24.]]) 

X: tensor([[ 9., 10., 11.],
        [15., 16., 17.]]) 
Y: tensor([[10., 11., 12.],
        [16., 17., 18.]]) 

X: tensor([[3., 4., 5.],
        [0., 1., 2.]]) 
Y: tensor([[4., 5., 6.],
        [1., 2., 3.]]) 



# 6.3.3.2 相邻采样

In [30]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_indices = torch.tensor(corpus_indices, dtype=torch.float32, device=device)
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size
    indices = corpus_indices[0: batch_size * batch_len].view(batch_size, batch_len)
    epoch_size = (batch_len - 1) // num_steps

    for i in range(epoch_size):
        i = i * num_steps
        X = indices[:, i: i + num_steps ]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y

In [31]:
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

UnboundLocalError: local variable 'batch_len' referenced before assignment